# Projet 5 - Segmentez des clients d'un site e-commerce - exploration de différents modèles de clustering

In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import psutil
import gc
import os 

def monitor_resources():
    process = psutil.Process(os.getpid())
    memory_mb = process.memory_info().rss / 1024 / 1024
    print(f"Mémoire: {memory_mb:.1f} MB")
    print(f"Mémoire système libre: {psutil.virtual_memory().available / 1024 / 1024:.1f} MB")

# Test avec données minimales
print("=== TEST AVEC DONNÉES MINIMALES ===")
try:
    from custom_library import outils
    
    # Créez un très petit dataset de test
    import pandas as pd
    import numpy as np
    
    # Dataset minimal pour le test
    test_data = pd.DataFrame({
        'feature1': np.random.rand(30000),  # Seulement 50 points !
        'feature2': np.random.rand(30000)
    })
    
    print(f"Taille dataset test: {test_data.shape}")
    monitor_resources()
    
    # Test DBSCAN avec paramètres conservateurs
    print("Test DBSCAN...")
    result_dbscan = outils.prepare_compute_evaluate_dbscan(
        test_data, 
        eps=0.5,  # Paramètres simples
        min_samples=5,
        data_description = "test"
    )
    monitor_resources()
    
    # Nettoyage mémoire entre les tests
    gc.collect()
    
    print("Test KMeans...")
    result_kmeans = outils.prepare_compute_evaluate_kmeans(
        test_data,
        max_num_clusters=3,  # Petit nombre de clusters
        data_description = "test",
        random_state = 42
    )
    monitor_resources()
    
except Exception as e:
    print(f"Erreur: {e}")
    monitor_resources()

=== TEST AVEC DONNÉES MINIMALES ===
Taille dataset test: (30000, 2)
Mémoire: 283.8 MB
Mémoire système libre: 12505.5 MB
Test DBSCAN...
# test
Avec les paramètres choisis, DBSCAN a défini 1 clusters
Il y a 0.0% d'outliers


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from math import pi

from sklearn.preprocessing import PowerTransformer
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, DBSCAN
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import silhouette_score, davies_bouldin_score, adjusted_rand_score

from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from hdbscan.validity import validity_index


from custom_library.outils import prepare_compute_evaluate_kmeans, prepare_compute_evaluate_dbscan

## Importation des données nettoyées

In [6]:
with open("clean_dataset.pkl","rb") as f:
	data = pd.read_pickle(f)

In [7]:
data.shape

(95082, 24)

In [4]:
data.columns

Index(['nb_commandes', 'nb_produits', 'depense_totale', 'recence',
       'score_moyen', 'cat_Alimentation & Boissons', 'cat_Animaux',
       'cat_Auto & Transport', 'cat_Autres', 'cat_Bricolage & Jardin',
       'cat_Bébé & Enfants', 'cat_Cuisine & Accessoires décoratifs',
       'cat_Loisirs & Culture', 'cat_Mobilier & Aménagement intérieur',
       'cat_Mode & Accessoires', 'cat_Papeterie & Bureau',
       'cat_Santé & Beauté', 'cat_Électroménager',
       'cat_Électronique & Informatique', 'cat_Centre-Ouest', 'cat_Nord',
       'cat_Nord-Est', 'cat_Sud', 'cat_Sud-Est'],
      dtype='object')

## Diviser en 4 différents jeux de données : 
- données RFM i.e. qui contiennent seulement les variables de récence ("recence"), de fréquence ("nb_commandes") et de montant ("depense_totale")
- données numériques i.e. qui contiennent seulement les variables numériques, et pas les variables catégorielles encodées
- données totales
- données numériques sans récence car elle n'a pas beaucoup de sens pour notre jeu de données sachant que 96% des clients n'ont passé qu'une seule commande. 

In [5]:
print(f"Avant d'enlever les valeurs manquantes, le dataset contenait {len(data)} clients")

data_RFM = data[['recence','nb_commandes','depense_totale']]
data_RFM.dropna(inplace=True)
print(f"Après avoir enlevé les valeurs manquantes du dataset data_RFM, il reste {len(data_RFM)} clients")

data_numeric = data.iloc[:,0:5]
data_numeric.dropna(inplace=True)
print(f"Après avoir enlevé les valeurs manquantes du dataset data_numeric, il reste {len(data_numeric)} clients")

data_num_WO_recency = data_numeric.drop('recence',axis = 1)
data_num_WO_recency.dropna(inplace=True)
print(f"Après avoir enlevé les valeurs manquantes du dataset data_num_WO_recency, il reste {len(data_num_WO_recency)} clients")

data_tot = data
data_tot.dropna(inplace=True)
print(f"Après avoir enlevé les valeurs manquantes du dataset data_tot, il reste {len(data_tot)} clients")

datasets_list = [data_RFM,data_numeric,data_num_WO_recency,data_tot]
datasets_descriptions = ["Variables RFM seules", "Variables numériques seules", "Variables numériques sans la variables récence", "Variables numériques et catégorielles encodées"]

Avant d'enlever les valeurs manquantes, le dataset contenait 95082 clients
Après avoir enlevé les valeurs manquantes du dataset data_RFM, il reste 95082 clients
Après avoir enlevé les valeurs manquantes du dataset data_numeric, il reste 94385 clients
Après avoir enlevé les valeurs manquantes du dataset data_num_WO_recency, il reste 94385 clients
Après avoir enlevé les valeurs manquantes du dataset data_tot, il reste 94385 clients


/tmp/ipykernel_3956/1580952926.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_RFM.dropna(inplace=True)


## Méthode du Kmeans

K-Means est un algorithme de **clustering** non supervisé qui cherche à partitionner les données en `k` groupes (ou "clusters") en minimisant la **variance intra-cluster**. Le processus se déroule en plusieurs étapes :

1. Initialisation de `k` centroïdes (aléatoirement ou par méthode heuristique).
2. Attribution de chaque point au centroïde le plus proche (selon la distance euclidienne).
3. Recalcul des centroïdes comme moyenne des points de chaque cluster.
4. Répétition des étapes 2 et 3 jusqu’à convergence (peu ou pas de changement dans les centroïdes).

Plusieurs **métriques** permettent d’évaluer la qualité du regroupement et d’aider à choisir un `k` pertinent :

1. L'inertie
- Mesure la variance intracluster
- Inertie minimale est signe de clusters denses

2. Méthode du coude (Elbow method)
- Repose sur la mesure de la **somme des distances au carré** entre les points et leur centroïde (inertie intra-cluster).
- On trace l'inertie en fonction de `k`. Le **"coude"** sur la courbe indique un bon compromis entre complexité du modèle et performance.

3. Coefficient de silhouette
- Mesure la **similarité d’un point avec son propre cluster** comparée à celui le plus proche.
- Le score varie entre -1 (mauvais regroupement) et 1 (regroupement optimal).
- Une moyenne élevée indique que les clusters sont bien séparés et denses.

4. Indice de Davies-Bouldin
- Évalue la **compacité et la séparation** des clusters.
- Repose sur le ratio entre l’étalement intra-cluster et la distance inter-cluster --> plus l’indice est **faible**, meilleure est la qualité du regroupement.



In [ ]:
for idx in range(len(datasets_list)):
	prepare_compute_evaluate_kmeans(datasets_list[idx], datasets_descriptions[idx], max_num_clusters = 8, random_state = 1)

La présence ou non de la variable récence dans le dataset composé des variables numériques seules n'a pas d'impact sur les résultats du kmeans. <br> L'ajout des variables encodées détériore fortement les résultats du kmeans. 
Le KMeans se base sur des distances euclidiennes, qui supposent une continuité et une échelle comparable entre les variables.
L’ajout de variables catégorielles encodées en one-hot peut déséquilibrer les distances :
- chaque catégorie introduit une nouvelle dimension binaire, ce qui augmente artificiellement l'influence de cette variable.
- cela peut déformer l’espace de distances et dégrader fortement la qualité des clusters.
Une solution pourrait être de changer la façon de calculer les distances en utilisant des équivalents au kmeans basées sur d'autres types de distances par exemple distance de manhattan. 

In [ ]:
k_choisi = 4
data_chosen = data_numeric

In [ ]:
# Application finale du K-means avec le K choisi
	# Préparation des données
pt = PowerTransformer(method="yeo-johnson", standardize = True)
X_transformed = pt.fit_transform(data_chosen)
final_kmeans = KMeans(n_clusters=k_choisi, random_state=42)
final_clusters = final_kmeans.fit_predict(X_transformed)

# Affichage des scores finaux
print(f"\nScores pour K = {k_choisi}:")
print(f"Inertie: {final_kmeans.inertia_}")
print(f"Score Silhouette: {silhouette_score(X_transformed, final_clusters)}")
print(f"Davies-Bouldin Index: {davies_bouldin_score(X_transformed, final_clusters)}")

- data_numeric, random state = 42 : <br> 
Scores pour K = 4:
    - Inertie: 79114.74181976692
    - Score Silhouette: 0.4485005102704839
    - Davies-Bouldin Index: 0.7360699869869303

<!-- - data_numeric, random state = 12 : <br>
Scores pour K = 4:
	- Inertie: 79114.54283500843
	- Score Silhouette: 0.4484665986665208
	- Davies-Bouldin Index: 0.7360158793413563

--> le modèle est assez stable -->

- data_RFM, random_state = 42: <br>
Scores pour K = 4:
	- Inertie: 10403
	- Score Silhouette: 0.54
	- Davies-Bouldin Index: 0.54

## Comparer les profils moyens des clusters sur chaque variable.

In [ ]:
# Centroïdes (dans l'espace standardisé si KMeans a été fait sur les données standardisées)
centroids = pd.DataFrame(final_kmeans.cluster_centers_, columns=data_chosen.columns)

# Barplot par cluster
centroids.T.plot(kind="bar", figsize=(12, 6))
plt.title("Centroïdes des clusters (standardisés)")
plt.ylabel("Valeur moyenne par cluster")
plt.xlabel("Variables")
plt.xticks(rotation=45, ha='right') 
plt.legend(title="Cluster")
plt.tight_layout()
plt.show()


In [ ]:
sns.heatmap(centroids.T, annot=False, cmap="coolwarm", center=0)
plt.title("Heatmap des centroïdes")
plt.ylabel("Variables")
plt.xlabel("Clusters")
plt.show()

## Visualiser le profil complet de chaque cluster sur un cercle

In [ ]:
def plot_radar(centroids_df, data_orig_df, labels):
    # Calcul des moyennes originales par cluster
    original_means = data_orig_df.copy()
    original_means["cluster"] = labels
    original_means = original_means.groupby("cluster").median()

    categories = centroids_df.columns.tolist()
    N = len(categories)
    angles = [n / float(N) * 2 * pi for n in range(N)]
    angles += angles[:1]

    plt.figure(figsize=(8, 8))

    for idx, row in centroids_df.iterrows():
        values = row.tolist()
        values += values[:1]
        plt.polar(angles, values, label=f"Cluster {idx}")

        # Afficher les vraies moyennes en annotation
        for j, var in enumerate(categories):
            angle = angles[j]
            radius = row[var]
            orig_val = original_means.loc[idx, var]
            offset = 0.05 if radius >= 0 else -0.05
            plt.text(
                angle,
                radius + offset,
                f"{orig_val:.2f}",
                ha="center",
                va="center",
                fontsize=10,
                color="black",
            )
        print(idx)

    plt.xticks(angles[:-1], categories, color="grey", size=12)
    plt.title("Radar des centroïdes (standardisés)")
    plt.legend(loc="upper right", bbox_to_anchor=(1.3, 1.1))
    plt.tight_layout()
    plt.show()


plot_radar(centroids_df=centroids, data_orig_df=data_chosen, labels=final_clusters)


#### data_RFM :
Les 4 clusters se distinguent seulement par leur dépense totale moyenne

#### data_numeric : 

- Cluster 0 : clients satisfaits, qui achètent peu mais des produits chers
- Cluster 1 : dépensiers moyens mais particulièrement insatisfaits
- Cluster 2 : parmi les plus dépensiers, qui achètent beaucoup de produits pas chers. Leur satisfaction est plutot moyenne
- Cluster 3 : clients économes et satisfaits


#### Conclusion 
La division en clusters à partir de data_numeric semble avoir plus de sens et porter plus d'informations métier que pour data_RFM. Malgré ses scores moindre, on choisit donc le dataset_numeric

## Comparer la distribution de chaque variable selon le cluster

In [ ]:
data_chosen["cluster"] = final_kmeans.labels_

# Pour chaque variable quantitative
for col in data_chosen.columns:
    plt.figure(figsize=(8, 4))
    sns.boxplot(x="cluster", y=col, data=data_chosen)
    plt.title(f"Distribution de {col} par cluster")
    plt.show()


## Visualiser les clusters en 2D grâce à une ACP

In [ ]:
pca = PCA(n_components=2)
components = pca.fit_transform(X_transformed)
pca_df = pd.DataFrame(components, columns=["PC1", "PC2"])
pca_df["cluster"] = final_kmeans.labels_

plt.figure(figsize=(8, 6))
sns.scatterplot(data=pca_df, x="PC1", y="PC2", hue="cluster", palette="Set2")
plt.title("ACP - Visualisation des clusters")
plt.xlabel(f"PC1 ({pca.explained_variance_ratio_[0] * 100:.1f}%)")
plt.ylabel(f"PC2 ({pca.explained_variance_ratio_[1] * 100:.1f}%)")
plt.legend(title="Cluster")
plt.show()


# DBSCAN 

DBSCAN (Density-Based Spatial Clustering of Applications with Noise) est un algorithme de clustering basé sur la densité.
Il regroupe les points densément connectés et identifie les points isolés comme du bruit --> le nombre de clusters n'est pas un paramètre, il émerge naturellement
Il nécessite deux paramètres :
- eps : la distance maximale entre deux points pour qu'ils soient considérés comme voisins.
- min_samples : le nombre minimum de points pour former un noyau dense (core point).

Fonctionnement :
1. Pour chaque point, DBSCAN récupère ses voisins dans un rayon eps.
2. Si le point a au moins 'min_samples' voisins, il devient un point noyau et forme un cluster.
3. Les voisins directs et indirects (par transitivité) sont ajoutés au cluster.
4. Les points trop isolés (pas assez de voisins) sont marqués comme bruit.
Avantages : détecte des clusters de forme arbitraire et gère bien le bruit.
Inconvénients : sensible au choix des paramètres et moins performant lorsque les densités sont très variables.

D'après [cette source](https://stackoverflow.com/questions/15050389/estimating-choosing-optimal-hyperparameters-for-dbscan/15063143#15063143), la valeur optimale de min_samples est 2xnb_features ici et la valeur optimale de eps est définie par un coude dans la courbe du classement croissant des distances des K plus proches voisins (K = 2xnb_features-1) : les distances augmentent d'un coup pour les points correspondant à du bruit. Le code ci-dessous permet de tracer cette courbe et de définir eps : 

In [ ]:
def get_kdist_plot(X=None, k=None, radius_nbrs=1.0):
    nbrs = NearestNeighbors(n_neighbors=k, radius=radius_nbrs).fit(X)

    # For each point, compute distances to its k-nearest neighbors
    distances, indices = nbrs.kneighbors(X)

    distances = np.sort(distances, axis=0)
    distances = distances[:, k - 1]

    # Plot the sorted K-nearest neighbor distance for each point in the dataset
    plt.figure(figsize=(8, 8))
    plt.plot(distances)
    plt.xlabel("Points/Objects in the dataset", fontsize=12)
    plt.ylabel("Sorted {}-nearest neighbor distance".format(k), fontsize=12)
    plt.grid(True, linestyle="--", color="black", alpha=0.4)
    plt.show()
    plt.close()


pt = PowerTransformer(method="yeo-johnson", standardize = True)
## Sur les variables numériques : 
X_transformed = pt.fit_transform(data_numeric)
k = 2 * X_transformed.shape[-1] - 1  # k=2*{dim(dataset)} - 1
get_kdist_plot(X=X_transformed, k=k)
## Sur toutes les variables
X_transformed = pt.fit_transform(data_tot)
k = 2 * X_transformed.shape[-1] - 1  # k=2*{dim(dataset)} - 1
get_kdist_plot(X=X_transformed, k=k)

D'après ces courbes, la valeur optimale de eps est : 
- 0.005 pour data_numeric  
- 1 pour data_tot

In [ ]:
clustering = DBSCAN(
    eps=0.5, min_samples=10, metric="euclidean", algorithm="ball_tree"
).fit(X_transformed)
clusters = clustering.labels_
print(f"Avec les paramètres choisis, DBSCAN a défini {len(set(clusters))} clusters")

In [ ]:
# Taux d'outliers
mask = (clusters == -1)
percent_outliers = np.round(len(X_transformed[mask]) / len(X_transformed) * 100, 4)
print(f"Il y a {percent_outliers}% d'outliers")

# Enlever les points marqués comme bruit (label = -1)
mask = clusters != -1
X_WO_outliers = X_transformed[mask]
labels_WO_outliers = clusters[mask]

# Score spécifique auxalgorithmes de densité
validity = validity_index(X_WO_outliers, labels_WO_outliers, metric="euclidean")
print(f"Density-based cluster validity : {validity}")

# Moyennes par cluster
unique_labels = np.unique(labels_WO_outliers)
cluster_means = []

for label in unique_labels:
    cluster_points = X_WO_outliers[labels_WO_outliers == label]
    cluster_means.append(cluster_points.mean(axis=0))

cluster_means = np.array(cluster_means)

# Radar plot
num_vars = X_WO_outliers.shape[1]
angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()
angles += angles[:1]  # fermeture du polygone


# Plot
fig, ax = plt.subplots(figsize=(8, 6), subplot_kw=dict(polar=True))

for i, row in enumerate(cluster_means):
    row_closed = np.concatenate([row, [row[0]]])
    ax.plot(angles, row_closed, label=f"Cluster {unique_labels[i]}")
    ax.fill(angles, row_closed, alpha=0.2)

# Labels des axes
feature_labels = [f"{data_chosen.columns[i]}" for i in range(num_vars)]
angles_labels = angles[:-1]  # enlever l'angle du doublon
ax.set_xticks(angles_labels)
ax.set_xticklabels(feature_labels)

ax.set_title("Profil moyen par cluster (DBSCAN)", y=1.08)
ax.legend(loc="upper right", bbox_to_anchor=(1.2, 1.1))
plt.tight_layout()
plt.show()


#### DBSCAN sur data_numeric

In [ ]:
prepare_compute_evaluate_dbscan(data = datasets_list[1], data_description = datasets_descriptions[1], eps = 0.005, min_samples = 2*len(datasets_list[1].columns))

# Variables numériques seules
Avec les paramètres choisis, DBSCAN a défini 359 clusters
Il y a 10.489% d'outliers


Avec ces valeurs de paramètres, on obtient 359 clusters. Autant de clusters empêche l'interprétation des résultats. On garde donc la valeur par défaut (0.5) qui donne 4 clusters :

In [ ]:
prepare_compute_evaluate_dbscan(data = datasets_list[1], data_description = datasets_descriptions[1], eps = 0.5, min_samples = 2*len(datasets_list[1].columns))

#### DBSCAN sur data_numeric

In [ ]:
prepare_compute_evaluate_dbscan(data = datasets_list[-1], data_description = datasets_descriptions[-1], eps = 1, min_samples = 2*len(datasets_list[1].columns))

# Clustering hiérarchique

Le clustering hiérarchique regroupe les points par similarité de manière progressive.
Il commence avec chaque point dans son propre cluster, puis fusionne les plus proches (approche agglomérative).
Le processus continue jusqu’à obtenir un seul cluster ou un nombre défini de clusters.
La proximité entre clusters peut être mesurée par différents critères (moyenne, minimum, Ward i.e. regrouppement entrainant la plus faible augmentation d'inertie).
Le résultat peut être visualisé sous forme de dendrogramme (arbre de fusions).
Il n’est pas nécessaire de définir le nombre de clusters à l’avance : on peut couper l’arbre à différents niveaux.
Peut créer des problèmes de mémoire RAM donc on fait un K means grossier en amont, afin de se ramener à 10 000 clusters (technique de réduction du nombre de lignes) 

peut créer des pb de mémoire RAM. K means grossier --> 1000 clusters = technique de réduction du nombre de lignes. Puis clustering hiérarchique sur ces clusters. Ensuite choisir le nb de cluster sur la base de silhouette et autre métriques utilisées sur k means.

In [ ]:
# kmeans préliminaire - réduction du nombre de lignes
preliminary_kmeans = KMeans(n_clusters=10_000, random_state=42)
preliminary_cluster_labels = preliminary_kmeans.fit_predict(X_transformed)

# silhouette_avg = silhouette_score(X_transformed, preliminary_cluster_labels)
# db_avg = davies_bouldin_score(X_transformed, preliminary_cluster_labels)
# inertia = preliminary_kmeans.inertia_

# print(
#     f"Inertie : {inertia} \nSilhouette : {silhouette_avg} \nDavies Bouldin : {db_avg}"
# )

In [ ]:
## Clustering hiérarchique par approche agglomérative

centroids = pd.DataFrame(
    preliminary_kmeans.cluster_centers_, columns=data_chosen.columns
)

#Calcul des liaisons hiérarchiques
Z = linkage(centroids, method="ward")

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 8))

_ = dendrogram(Z, p=10, truncate_mode="lastp", ax=ax)

plt.title("Hierarchical Clustering Dendrogram")
plt.xlabel("Number of points in node (or index of point if no parenthesis).")
plt.ylabel("Distance.")
plt.show()

In [ ]:
for k in range(2, 11):
    labels = fcluster(Z, k, criterion="maxclust")
    silhouette_avg = silhouette_score(preliminary_kmeans, labels)
    db_avg = davies_bouldin_score(preliminary_kmeans, labels)
    print(f"Silhouette : {silhouette_avg} \nDavies Bouldin : {db_avg}")
    score = silhouette_score(X, labels)
    print(score)

Modifier le code ci dessus pour attribuer le cluster à chaque objet issu du kmeans. 
On peut aussi évaluer la cohérence de l'arbre pour choisir le nombre de clusters --> implémenté ici : https://docs.scipy.org/doc/scipy-1.15.2/reference/generated/scipy.cluster.hierarchy.inconsistent.html Il existe d'autres méthodes appropriées à l'arbre : https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.cut_tree.html

Critère de choix du modèle : l'interprétabilité et stabilité. 